In [17]:
import pandas as pd
import json
import numpy as np
import math
import heapq

total_link = pd.read_csv("C:/Users/gimm7/OneDrive/바탕 화면/modified_link.csv")
total_node = pd.read_csv("C:/capstone_design2_data/total_node.csv")

def compose_coordinate(latitude, longitude):
    composed_coordinate = '['+ str(longitude)+ ','+ str(latitude)+ ']'
    return composed_coordinate

# def haversine_distance(coord1, coord2):
#     R = 6371.0  # Earth's radius in km
#     lat1 = math.radians(coord1[0])
#     lon1 = math.radians(coord1[1])
#     lat2 = math.radians(coord2[0])
#     lon2 = math.radians(coord2[1])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     distance = R * c
#     return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c

    #distance = ((lat1-lat2)**2+(lon1-lon2)**2)**0.5



    return distance


# 시작점 입력

In [14]:
dep_lat, dep_lon = 37.294195879066635, 127.0033176194018
des_lat, des_lon = 37.29475413961952, 127.00458455362698

# dep_lat = 37.28293695757347
# dep_lon = 127.0434230405271
# des_lat = 37.28312042096528
# des_lon = 127.04739397575078

# Get Nearest Node

In [15]:
def get_Nearest_Node(latitude, longitude):
    length_filter = 0.0003
    min_distance = 1e9
    nearest_lat = 0
    nearest_lon = 0
    nearest_idx = 0
    
    filtered_df = total_node.loc[(total_node['위도'] > latitude - length_filter) & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] > longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]
    for index, row in filtered_df.iterrows():

        cur_node_lat = row['위도']
        cur_node_lon = row['경도']
        cur_distance = haversine_distance(latitude, longitude, cur_node_lat, cur_node_lon)

        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_lat = cur_node_lat
            nearest_lon = cur_node_lon
            nearest_idx = index

    #return filtered_df
    return nearest_idx, nearest_lat, nearest_lon
    

start_idx, start_lat, start_lon = get_Nearest_Node(dep_lat, dep_lon)
end_idx, end_lat, end_lon = get_Nearest_Node(des_lat, des_lon)


In [8]:
print(start_idx, end_idx)

416241 407660


# 다익스트라 알고리즘

In [18]:
area_N = max(dep_lat, des_lat) + 0.005
area_S = min(dep_lat, des_lat) - 0.005
area_E = max(dep_lon, des_lon) + 0.005
area_W = min(dep_lon, des_lon) - 0.005

areal_link = total_link.loc[(total_link['노드1위도'] < area_N)
                             & (total_link['노드2위도'] < area_N)
                             & (total_link['노드1위도'] > area_S) 
                             & (total_link['노드2위도'] > area_S)
                             & (total_link['노드1경도'] < area_E)
                             & (total_link['노드2경도'] < area_E)
                             & (total_link['노드1경도'] > area_W)
                             & (total_link['노드2경도'] > area_W)]

In [19]:
areal_link

,Unnamed: 0,idx노드1,idx노드2,노드1경도,노드1위도,노드2경도,노드2위도,노드1,노드2,weight,...,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
32466,32466,36079.0,36080.0,126.998672,37.290326,126.998629,37.290308,"[126.99867183768276, 37.29032589548867]","[126.99862884369283, 37.29030784243086]",0.004301,...,0,0,0,0,0,0,0,0,0,0
32467,32467,36080.0,36081.0,126.998629,37.290308,126.998370,37.290195,"[126.99862884369283, 37.29030784243086]","[126.99837036428146, 37.29019515433979]",0.026074,...,0,0,0,0,0,0,0,0,0,0
32469,32469,36083.0,36079.0,126.998745,37.290357,126.998672,37.290326,"[126.99874520371186, 37.29035670201854]","[126.99867183768276, 37.29032589548867]",0.007339,...,0,0,0,0,0,0,0,0,0,0
44483,44483,51641.0,51642.0,127.008085,37.289201,127.008061,37.289210,"[127.00808524170385, 37.2892009648496]","[127.0080614867841, 37.28920998999186]",0.002329,...,0,0,1,0,0,0,0,0,0,1
44484,44484,51642.0,51643.0,127.008061,37.289210,127.008059,37.289211,"[127.0080614867841, 37.28920998999186]","[127.00805875819219, 37.28921130549739]",0.000282,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335140,335144,428377.0,356231.0,127.004900,37.298167,127.005061,37.298351,"[127.00490005569006, 37.298167319771686]","[127.00506082174694, 37.29835083703431]",0.024872,...,0,0,0,0,0,5,0,0,0,0
335141,335145,407984.0,428380.0,127.003824,37.297903,127.003882,37.297968,"[127.00382412382375, 37.29790251825934]","[127.00388192986635, 37.297968363640166]",0.008930,...,0,0,0,0,0,9,0,0,0,0
335142,335146,428380.0,357045.0,127.003882,37.297968,127.003939,37.298034,"[127.00388192986635, 37.297968363640166]","[127.00393930813406, 37.298033731266244]",0.008865,...,0,0,0,0,0,7,0,0,0,0
335143,335147,357076.0,428383.0,127.003526,37.298034,127.003624,37.297978,"[127.00352610359054, 37.29803423230363]","[127.00362364946791, 37.29797801262323]",0.010655,...,0,0,0,0,0,9,0,0,0,0
